###### CSCE 320 :: Principles of Data Science ::Texas A&M University :: Fall 2020


# Group Project: Analysis of Election Data over Time for the purpose of finding the Optimal set of Training Data



Members: Matthew Spence, Adrian Shi, Marcel Theo, Michael Chaco

## Major Question:
What chronological subset of training data produces the most accurate election forecasting models?

## Purpose:
The outcomes of presidential elections shape U.S. government policy, which affects the lives of every American. We hope to determine if a predictive model generated from U.S. polling data is more or less accurate than historical data. 
Potential beneficiaries of this project are U.S. allies or businesses looking to predict the types of policies that will be implemented (which can affect their business)

## Method:
We intend to train two machine learning models of the same type on two different sets of training data. One will be trained on data going back as far as is available, and the other will be trained only on recent data. We will then compare the predictions of the two models to the actual outcome of the 2020 election, and see which is more accurate. 
 
### Getting started: 
First we need to gather our data and clean it. Our datasets are as follows: 
Popular Vote in U.S. Elections (Statista)
Historical Electoral Vote (Data Galore)
Historic head to head polling data (Gallup)
Incumbent approval ratings (Gallup)
Past electoral votes by state (Electoral-Vote.com)
Government Approval Ratings (Pew Research)
Unemployment rate (Trading Economics)
GDP growth by year (Trading Economics)


In [20]:
# imports
import pandas as pd
import matplotlib.pyplot as plt

# insert imports and data cleaning here
elections = pd.read_csv("CSV/elections-1900-to-2012.csv", encoding="utf-8") 
electoralVotes = pd.read_csv("CSV/electoral_votes.csv", encoding="utf-8") 
approvalRatings = pd.read_csv("CSV/incumbency_approval_ratings.csv", encoding="utf-8") 
#polling = pd.read_csv("CSV/polling.csv", encoding="utf-8") 
popularVotes = pd.read_csv("CSV/popular_votes.csv", encoding="utf-8") 
trustRating = pd.read_csv("CSV/trust_rating.csv", encoding="utf-8")

CleanedData = pd.read_csv("CSV/cleaned_data.csv", encoding="utf-8")

#change percentages in CleanedData to decimal
CleanedData['poll_d'] = CleanedData['poll_d'].str.rstrip('%').astype('float') / 100.0
CleanedData['poll_r'] = CleanedData['poll_r'].str.rstrip('%').astype('float') / 100.0
CleanedData['gdp_percent'] = CleanedData['gdp_percent'].str.rstrip('%').astype('float') / 100.0
CleanedData['electoral_r_percent'] = CleanedData['electoral_r_percent'].str.rstrip('%').astype('float') / 100.0
CleanedData['electoral_d_percent'] = CleanedData['electoral_d_percent'].str.rstrip('%').astype('float') / 100.0
CleanedData['popular_d_percent'] = CleanedData['popular_d_percent'].str.rstrip('%').astype('float') / 100.0
CleanedData['popular_r_percent'] = CleanedData['popular_r_percent'].str.rstrip('%').astype('float') / 100.0

CleanedData.head()

,year,electoral_d,electoral_r,total_electoral,electoral_d_percent,electoral_r_percent,popular_d,popular_r,popular_d_percent,popular_r_percent,poll_d,poll_r,incumbent,incumbent_won,incumbent_approval,trust,gdp_percent,current_party,party_won
0,1900,155.0,292.0,447.0,0.3468,0.6532,6370932.0,7228864.0,0.4550,0.5170,NaN,NaN,0,0.0,NaN,NaN,NaN,1,1.0
1,1904,140.0,336.0,476.0,0.2941,0.7059,5083880.0,7228864.0,0.3760,0.5640,NaN,NaN,1,1.0,NaN,NaN,NaN,1,1.0
2,1908,162.0,321.0,483.0,0.3354,0.6646,6408979.0,7678335.0,0.4300,0.5160,NaN,NaN,0,-1.0,NaN,NaN,NaN,1,1.0
3,1912,435.0,8.0,531.0,0.8192,0.0151,6296284.0,3486242.0,0.4180,0.2320,NaN,NaN,1,0.0,NaN,NaN,NaN,1,0.0
4,1916,277.0,254.0,531.0,0.5217,0.4783,9126868.0,8548728.0,0.5216,0.4783,NaN,NaN,1,1.0,NaN,NaN,NaN,0,0.0


### A key decision in this project will be what machine learning model we use. We intend to use a model from the sklearn library. The goal of this project is not to test the accuracy of a given model per se, but rather to find the best set of training data. As such we considered a few relatively simple models:
Linear Regression<br>
Ridge Regression<br>
Lasso Regression<br>

### After reviewing the current techniques used in professional election forecasting, we decided that a linear regression would serve our purposes well. This model is already familiar to us through our class work, and is easy to work with. This will enable us to spend less time worrying about optimizing our model and more time testing different subsets of data. 



In [21]:
# get the data into a dataframe
data = pd.DataFrame(CleanedData)
data.head()

,year,electoral_d,electoral_r,total_electoral,electoral_d_percent,electoral_r_percent,popular_d,popular_r,popular_d_percent,popular_r_percent,poll_d,poll_r,incumbent,incumbent_won,incumbent_approval,trust,gdp_percent,current_party,party_won
0,1900,155.0,292.0,447.0,0.3468,0.6532,6370932.0,7228864.0,0.4550,0.5170,NaN,NaN,0,0.0,NaN,NaN,NaN,1,1.0
1,1904,140.0,336.0,476.0,0.2941,0.7059,5083880.0,7228864.0,0.3760,0.5640,NaN,NaN,1,1.0,NaN,NaN,NaN,1,1.0
2,1908,162.0,321.0,483.0,0.3354,0.6646,6408979.0,7678335.0,0.4300,0.5160,NaN,NaN,0,-1.0,NaN,NaN,NaN,1,1.0
3,1912,435.0,8.0,531.0,0.8192,0.0151,6296284.0,3486242.0,0.4180,0.2320,NaN,NaN,1,0.0,NaN,NaN,NaN,1,0.0
4,1916,277.0,254.0,531.0,0.5217,0.4783,9126868.0,8548728.0,0.5216,0.4783,NaN,NaN,1,1.0,NaN,NaN,NaN,0,0.0


### Once we have all of our raw data in an easily usable format, we need to divide the data into testable subsets. We want to see what chronologically continuous subset of training data provides the most accurate predictions for the 2020 election. That is, we want to train a model for every possible subset of data that includes every election after a given cutoff year, where the cutoff year will be set at some past election eg 1960-2016, 1964-2016, 1968-2016 ... 2012-2016, only 2016. This means that for n elections for which we have viable data, we will have n different subsets of data to test. 

In [22]:
# dataInRange returns the cleaned data in a given range of years.

def dataInRange(df, startYear, endYear):
    newDf = df[df['year'].between(startYear, endYear)]
    return newDf

### Before we train our models, we need the best available data on the 2020 election to get the various model's predictions.  

In [23]:
# import 2020 data
thisYear = data[data['year'].between(2020, 2020)]

### Now that we have our training data and our model, we need to decide what technique to use to train the model. We also need to decide what metrics to attempt to predict to measure the accuracy of the models. We start by simply predicting the binary outcome, and naively training the model on the test data. 

In [38]:
# train the models to predict the binary outcome.
from sklearn.linear_model import LinearRegression
import numpy as np

FIRST_YEAR = 1936
LAST_YEAR = 2016

predictions = []
    
# Iterate over every possible range of years
for i in range(FIRST_YEAR,LAST_YEAR, 4): 
        # train model for each of these subsets..
        subset = dataInRange(data, i, LAST_YEAR)
        x = subset[['poll_d', 'poll_r', 'incumbent', 'incumbent_approval', 'gdp_percent', 'current_party']]
        y = subset['popular_d_percent'] # binary outcome data from this subset of years
        model = LinearRegression()
        model.fit(x,y)
        
        ### Next, predict y from the 2020 data
        x_new = thisYear[['poll_d', 'poll_r', 'incumbent', 'incumbent_approval', 'gdp_percent', 'current_party']]
        y_new = model.predict(x_new)
        predictions.append(y_new[0]) # for some reason y_new creates an array of one element, so use the first (and only) element
        print(y_new)
        ### plot the results here?
        #plt.figure(figsize=(4, 3))
        #ax = plt.axes()
        #ax.scatter(x, y)
        #ax.plot(x_new, y_new)
        #ax.set_xlabel('x')
        #ax.set_ylabel('y')
        #ax.axis('tight')
        #plt.show()
        
print(predictions)
for i in range(len(predictions)):
    if predictions[i] < 0.50:
        predictions[i] = 1 #republicans won
    else:
        predictions[i] = 0 #democrats won

print(predictions)

[0.45394027]
[0.46009907]
[0.46457365]
[0.47121128]
[0.47173227]
[0.46638599]
[0.46567456]
[0.46625235]
[0.46959921]
[0.45568934]
[0.44561331]
[0.45080457]
[0.53503524]
[0.55061066]
[0.52274386]
[0.46758632]
[0.45621584]
[0.50147841]
[0.55685439]
[0.5110298]
[0.4539402748276713, 0.46009906672164635, 0.4645736478139863, 0.4712112812726697, 0.4717322679315341, 0.4663859905542392, 0.46567456368999255, 0.4662523499950911, 0.46959920688093737, 0.45568933808087986, 0.4456133143153954, 0.45080457259621365, 0.5350352416799797, 0.5506106556550199, 0.5227438629265304, 0.467586316446236, 0.4562158370739497, 0.501478411419391, 0.556854387688487, 0.5110297980027819]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0]


### We should now have our predictions, time to visualize the results

In [39]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import PIL

#predictions = [1,0,1,0,1,0,1,0,1]
images = []
redMap = PIL.Image.open('Images/RedMap.png')
blueMap= PIL.Image.open('Images/BlueMap.png')

for res in predictions:
    images.append(blueMap.copy() if res==1 else redMap.copy())

def display_grid(imgs, grid_shape, scale=0.1, axes_pad=0.07):

    # Grid must be 2D:
    assert len(grid_shape) == 2

    # Make sure all images can fit in grid:
    assert grid_shape[0]*grid_shape[1] >= len(imgs)

    grid = ImageGrid(plt.gcf(), 111, grid_shape, axes_pad=axes_pad)
    for i, data in enumerate(imgs):

        # Scale image:
        im = data
        thumb_shape = [im.size[0]*scale, im.size[1]*scale]
        im.thumbnail(thumb_shape, PIL.Image.ANTIALIAS)
        data_thumb = np.array(im)
        grid[i].imshow(data_thumb)

        # Turn off axes:
        grid[i].axes.get_xaxis().set_visible(False)
        grid[i].axes.get_yaxis().set_visible(False)
        grid[i].axis("off")

display_grid(images, [3, 3], scale=0.5)


AssertionError: 

### This gets us our predictions, but we have a problem. (Most/All/A lot) of the models predict the same result.The binary outcome is simply not enough to guage which model is most accurate. As such, we decided to predict the popular vote instead. This will allow for more granular comparisons, since popular votes are much more variable than binary election outcomes.  

In [ ]:
# use the same techniques as before the predict the popular vote margin

### Now that the 2020 election is over, we can compare the predictions of our models to the actual results. 

## Results: 